## Setup

In [ ]:
!pip install tensorflow==2.7.0
!pip install tensorflow-quantum
!pip install paddle==1.0.2
!pip install paddle_quantum==2.2.0
!pip install paddlepaddle==2.3.0
!pip install mindquantum==0.6.0
!pip install mindspore==1.6.1
!pip install pyqpanda==3.7.13
!pip install cirq==0.14.1
!pip install qiskit==0.36.2
!pip install qiskit_optimization==0.3.2
!pip install PennyLane==0.23.1

## MindQuantum benchmark function:

In [ ]:
from mindquantum.core import Circuit, Hamiltonian
from mindquantum.simulator import Simulator
from mindquantum.algorithm.nisq.qaoa import MaxCutAnsatz
from mindquantum.framework import MQAnsatzOnlyLayer
import mindspore.nn as nn
import mindspore as ms


def benchmark_mq(hyperparams={}):
    """
    Performs QAOA optimizations.

    Args:
            hyperparams (dict): hyperparameters to configure this benchmark

                    * 'graph': Graph represented as a NetworkX Graph class

                    * 'n_layers': Number of layers in the QAOA circuit

                    * 'params': Numpy array of trainable parameters that is fed into the circuit

                    * 'shots': The number of samples.
    """

    graph = hyperparams['graph']
    n_layers = hyperparams['n_layers']
    shots = hyperparams['shots']

    # 生成对应的的量子线路和哈密顿量
    maxcut = MaxCutAnsatz(list(graph.edges), n_layers)

    circ = maxcut.circuit  # 已对所有量子比特作用H门
    ham = Hamiltonian(-maxcut.hamiltonian)
    # print(circ)
    # print(ham)

    # 搭建待训练量子神经网络以获取最优参数
    ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")

    sim = Simulator('projectq', circ.n_qubits)

    grad_ops = sim.get_expectation_with_grad(ham, circ)  # 获取计算变分量子线路的期望值和梯度的算子

    net = MQAnsatzOnlyLayer(grad_ops)  # 生成待训练的神经网络
    opti = nn.Adam(net.trainable_params(), learning_rate=0.05)  # 设置针对网络中所有可训练参数、学习率为0.05的Adam优化器
    train_net = nn.TrainOneStepCell(net, opti)  # 对神经网络进行一步训练

    steps = hyperparams['iter_num']
    for i in range(steps):
        cut = (len(graph.edges) - train_net()) / 2  # 将神经网络训练一步并计算得到的结果（切割边数）。注意：每当'train_net()'运行一次，神经网络就训练了一步
        if i % 10 == 0:
            print("train step:", i, ", cut:", cut)  # 每训练10步，打印当前训练步数和当前得到的切割边数

    params_dict = dict(zip(circ.params_name, net.weight.asnumpy()))  # 获取训练得到的最优参数
    # print(params_dict)

    # 测量
    circ.measure_all()  # 为线路中所有比特添加测量门
    result = sim.sampling(circ, pr=params_dict, shots=shots)
    print("Measurement result with MindQuantum:\n", result)


## Paddle benchmark function:

In [ ]:
# import paddle
# from paddle_quantum.ansatz import Circuit
# from paddle_quantum.loss import ExpecVal
# from paddle_quantum import Hamiltonian
# import warnings
# warnings.filterwarnings("ignore")


# def circuit_QAOA(n_qubits, n_layers, edges, nodes):
#     # 初始化 n 个量子比特的量子电路
#     circ = Circuit(n_qubits)
#     # 制备量子态 |s>
#     circ.superposition_layer()
#     # 搭建 n_layers 层 ansatz 线路
#     circ.qaoa_layer(edges, nodes, n_layers)

#     return circ


# def benchmark_paddle(hyperparams={}):
#     """
#     Performs QAOA optimizations.

#     Args:
#             hyperparams (dict): hyperparameters to configure this benchmark

#                     * 'graph': Graph represented as a NetworkX Graph class

#                     * 'n_layers': Number of layers in the QAOA circuit

#                     * 'shots': The number of samples.
#     """

#     graph = hyperparams['graph']
#     n_layers = hyperparams['n_layers']
#     shots = hyperparams['shots']

#     edges = list(graph.edges)
#     nodes = list(graph.nodes)
#     n_qubits = len(graph.nodes)

#     circ = circuit_QAOA(n_qubits, n_layers, edges, nodes)

#     SEED = 1024  # 设置全局随机数种子
#     paddle.seed(SEED)

#     LR = 0.05  # 梯度下降的学习率
#     opt = paddle.optimizer.Adam(learning_rate=LR, parameters=circ.parameters())  # 使用 Adam 优化器

#     # 以 list 的形式构建哈密顿量
#     ham_list = []
#     for (u, v) in edges:
#         ham_list.append([1.0, 'z' + str(u) + ',z' + str(v)])
#     # print(ham_list)

#     # 构造损失函数
#     loss_func = ExpecVal(Hamiltonian(ham_list))

#     # 训练
#     steps = hyperparams['iter_num']
#     for i in range(steps):
#         state = circ()
#         # 计算梯度并优化
#         loss = loss_func(state)
#         loss.backward()
#         opt.minimize(loss)
#         opt.clear_grad()
#         if (i + 1) % 10 == 0:
#             print("training step:", i + 1, "  loss:", "%.4f" % loss.numpy())

#     # 测量
#     state = circ()
#     probs_measured = state.measure(shots=shots, plot=False)
#     print("Probability distribution of measurement result with PaddleQuantum:\n", probs_measured)


## QPanda benchmark function:

In [ ]:
import numpy as np
from pyqpanda import *


def qaoa_layer(qubit_list, Hamiltonian, beta, gamma):
    vqc = VariationalQuantumCircuit()
    for i in range(len(Hamiltonian)):
        tmp_vec = []
        item = Hamiltonian[i]
        dict_p = item[0]
        for iter in dict_p:
            if 'Z' != dict_p[iter]:
                pass
            tmp_vec.append(qubit_list[iter])

        coef = item[1]

        if 2 != len(tmp_vec):
            pass

        vqc.insert(VariationalQuantumGate_CNOT(tmp_vec[0], tmp_vec[1]))
        vqc.insert(VariationalQuantumGate_RZ(tmp_vec[1], 2 * gamma * coef))
        vqc.insert(VariationalQuantumGate_CNOT(tmp_vec[0], tmp_vec[1]))

    for j in qubit_list:
        vqc.insert(VariationalQuantumGate_RX(j, 2.0 * beta))
    return vqc


def benchmark_qpanda(hyperparams={}):
    """
    Performs QAOA optimizations.

    Args:
            hyperparams (dict): hyperparameters to configure this benchmark

                    * 'graph': Graph represented as a NetworkX Graph class

                    * 'n_layers': Number of layers in the QAOA circuit

                    * 'shots': The number of samples.
    """

    graph = hyperparams['graph']
    n_layers = hyperparams['n_layers']
    shots = hyperparams['shots']

    edges = list(graph.edges)
    n_qubits = len(graph.nodes)

    problem_ham = {}
    for (u, v) in edges:
        k = 'Z' + str(u) + ' Z' + str(v)
        if k in problem_ham:
            problem_ham[k] += 1.0
        else:
            problem_ham[k] = 1.0

    ham = PauliOperator(problem_ham)

    beta = var(np.ones((n_layers, 1), dtype='float64'), True)
    gamma = var(np.ones((n_layers, 1), dtype='float64'), True)

    machine = init_quantum_machine(QMachineType.CPU)
    qubit_list = machine.qAlloc_many(n_qubits)
    vqc = VariationalQuantumCircuit()

    for i in qubit_list:
        vqc.insert(VariationalQuantumGate_H(i))

    for i in range(n_layers):
        vqc.insert(qaoa_layer(qubit_list, ham.toHamiltonian(1), beta[i], gamma[i]))

    loss = qop(vqc, ham, machine, qubit_list)  # 问题哈密顿量的期望
    optimizer = MomentumOptimizer.minimize(loss, 0.02, 0.9)  # 使用梯度下降优化器来优化参数

    leaves = optimizer.get_variables()

    steps = hyperparams['iter_num']
    for i in range(steps):
        optimizer.run(leaves, 0)
        loss_value = optimizer.get_loss()
        if (i + 1) % 10 == 0:
            print("training step:", i + 1, "  loss:", "%.4f" % loss_value)

    # 验证结果
    prog = QProg()
    qcir = vqc.feed()
    prog.insert(qcir)
    directly_run(prog)

    result = quick_measure(qubit_list, shots)
    print("Measurement result with QPanda:\n", result)


## Pennylane benchmark function:

In [ ]:
import pennylane as qml
from pennylane import qaoa
import random


def to_bin_str(num, n):
    """Octal to binary conversion.

    Returns:
        binary string with length n.
    """
    binary_str = bin(num)[2:].zfill(n)
    return binary_str


def benchmark_pennylane(hyperparams={}):
    """
    Performs QAOA optimizations.

    Args:
        hyperparams (dict): hyperparameters to configure this benchmark

                * 'graph': Graph represented as a NetworkX Graph class

                * 'n_layers': Number of layers in the QAOA circuit

                * 'params': Numpy array of trainable parameters that is fed into the circuit

                * 'shots': The number of samples.
    """

    graph = hyperparams['graph']
    n_layers = hyperparams['n_layers']
    shots = hyperparams['shots']

    n_wires = len(graph.nodes)
    wires = range(n_wires)

    device = qml.device("default.qubit", wires=n_wires, shots=shots)

    cost_h, mixer_h = qaoa.maxcut(graph)
    # print("Cost Hamiltonian", cost_h)
    # print("Mixer Hamiltonian", mixer_h)

    def qaoa_layer(gamma, alpha):
        qaoa.cost_layer(gamma, cost_h)
        qaoa.mixer_layer(alpha, mixer_h)

    def circuit(params):
        for i in wires:
            qml.Hadamard(wires=i)
        qml.layer(qaoa_layer, n_layers, params[0], params[1])

    @qml.qnode(device)
    def cost_function(params):
        circuit(params)
        return qml.expval(cost_h)

    @qml.qnode(device)
    def probability_circuit(gamma, alpha):
        circuit([gamma, alpha])
        return qml.probs(wires=wires)

    if 'params' in hyperparams:
        params = hyperparams['params']
    else:
        single_param_for_all_layers = [random.random() for i in range(n_layers)]
        params = qml.numpy.array([single_param_for_all_layers, single_param_for_all_layers], requires_grad=True)  # 将每个初始参数设置为 0.5
        optimizer = qml.GradientDescentOptimizer()
        steps = hyperparams['iter_num']
        for i in range(steps):
            params = optimizer.step(cost_function, params)

        # print("Optimal Parameters: \n", params)

    probs_measured = probability_circuit(params[0], params[1])

    result_dict = dict()
    for i in range(2 ** n_wires):
        result_dict[to_bin_str(i, n_wires)] = probs_measured[i].numpy()

    print("Measurement result with PennyLane: \n", result_dict)


## Qiskit benchmark function:

In [ ]:
from qiskit import Aer
from qiskit_optimization.applications import Maxcut
from qiskit.utils import algorithm_globals
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
import numpy as np


def benchmark_qiskit(hyperparams={}):
    """
    Performs QAOA optimizations.

    Args:
            hyperparams (dict): hyperparameters to configure this benchmark

                    * 'graph': Graph represented as a NetworkX Graph class

                    * 'n_layers': Number of layers in the QAOA circuit

                    * 'shots': The number of samples.
    """

    graph = hyperparams['graph']
    n_layers = hyperparams['n_layers']
    shots = hyperparams['shots']

    n_qubits = len(graph.nodes)
    edges = list(graph.edges) 

    # Computing the weight matrix from the random graph
    adjacent_matrix = np.zeros([n_qubits, n_qubits])
    for u, v in edges:
        adjacent_matrix[u][v] += 1
    # print(adjacent_matrix)

    # Mapping to the Ising problem
    max_cut = Maxcut(adjacent_matrix)
    qp = max_cut.to_quadratic_program()
    print(qp.export_as_lp_string())

    # get the corresponding Ising Hamiltonian
    qubit_op, offset = qp.to_ising()
    # print("Offset:", offset)
    # print("Ising Hamiltonian:\n_qubits", str(qubit_op))

    algorithm_globals.random_seed = 10598

    optimizer = COBYLA()
    qaoa = QAOA(optimizer, reps=n_layers, quantum_instance=Aer.get_backend('statevector_simulator'))

    result = qaoa.compute_minimum_eigenvalue(qubit_op)
    print(result)

    most_likely_state = max_cut.sample_most_likely(result.eigenstate)
    print("Measurement result with Qiskit:\n", most_likely_state)
    print(f'The max number of crossing edges computed by QAOA is {qp.objective.evaluate(most_likely_state)}')


## Tensorflow Quantum benchmark function:

In [ ]:
import tensorflow as tf
import tensorflow_quantum as tfq
from mindquantum.algorithm.nisq.qaoa import MaxCutAnsatz
import cirq

import sympy
import numpy as np
import pandas as pd


def trans_hamiltonian(mq_hamiltonion, qreg):
    gate_map = {
        "X": cirq.ops.X,
        "Y": cirq.ops.Y,
        "Z": cirq.ops.Z,
    }
    ham = cirq.PauliSum()
    for term in mq_hamiltonion.terms:
        coef = float(mq_hamiltonion.terms[term])  # for mindquantum==0.6.0
        # coef = float(mq_hamiltonion.terms[term].const)  # for mindquantum==0.6.2

        if len(term) == 0:
            ham += coef
            continue

        v = []
        for op in term:
            g = gate_map[op[1]]
            idx = int(op[0])
            v.append(g.on(qreg[idx]))

        ham += coef * cirq.PauliString(*tuple(v))

    return ham


def qaoa_circuit(graph, qreg, layer: int, pr_table):
    # Symbols for the rotation angles in the QAOA circuit.
    alpha = sympy.Symbol(f"alpha_{layer}")
    beta = sympy.Symbol(f"beta_{layer}")

    circ = cirq.Circuit(
        # Prepare uniform superposition on working_qubits == working_graph.nodes
        cirq.H.on_each(qreg),
        # Do ZZ operations between neighbors u, v in the graph.
        (
            cirq.ZZ(qreg[u], qreg[v]) ** alpha
            for (u, v) in graph.edges()
        ),
        # Apply X operations along all nodes of the graph.
        cirq.Moment(cirq.X(qubit) ** beta for qubit in qreg)
    )
    pr_table[f"alpha_{layer}"] = alpha
    pr_table[f"beta_{layer}"] = beta
    # print(pr_table)
    return circ


def benchmark_tfq(hyperparams={}):
    """
    Performs QAOA optimizations.

    Args:
            hyperparams (dict): hyperparameters to configure this benchmark

                    * 'graph': Graph represented as a NetworkX Graph class

                    * 'n_layers': Number of layers in the QAOA circuit

                    * 'shots': The number of samples.

                    * 'iter_num': The number of iterations
    """

    graph = hyperparams['graph']
    n_layers = hyperparams['n_layers']
    shots = hyperparams['shots']

    n_qubits = len(graph.nodes)
    qreg = cirq.LineQubit.range(n_qubits)

    # Construct the QAOA circuit
    total_circuit = cirq.Circuit()
    pr_table = dict()
    for i in range(n_layers):
        total_circuit.append(qaoa_circuit(graph, qreg, i, pr_table))

    # All relevant things can be computed in the computational basis.
    total_circuit.append(cirq.measure(qubit) for qubit in qreg)

    # 生成对应的的量子线路和哈密顿量
    maxcut = MaxCutAnsatz(list(graph.edges), n_layers)
    ham_mq = -maxcut.hamiltonian
    print("hamiltonian in mq:\n", ham_mq)

    # transform mindquantum hamiltonian to cirq
    ham = trans_hamiltonian(ham_mq, qreg)
    print("hamiltonian in cirq:\n", ham)

    print("circuit in mq:")
    print(maxcut.circuit)
    print("circuit in cirq:")
    print(total_circuit)

    expectation_calculation = tfq.layers.Expectation(
        differentiator=tfq.differentiators.ForwardDifference(grid_spacing=0.01)
    )

    theta = np.zeros((1, len(pr_table))).astype(np.float32)
    theta_tensor = tf.convert_to_tensor(theta)

    steps = hyperparams['iter_num']
    for i in range(steps):
        with tf.GradientTape() as g:
            g.watch(theta_tensor)
            output = expectation_calculation(
                total_circuit,
                operators=ham,
                symbol_names=list(pr_table.keys()),
                symbol_values=theta_tensor,
            )
            grad = g.gradient(output, theta_tensor)
            theta_tensor -= grad
            if (i + 1) % 10 == 0:
                print("training step:", i + 1, "  loss:", "%.4f" % output[0].numpy())

            # if i == steps - 1:
            #   print(output)
            #   print(theta_tensor)

    params = theta_tensor.numpy()
    print(params)

    pr_list = list(pr_table.keys())

    for i in range(len(pr_list)):
        pr_table[pr_list[i]] = params[0][i]

    print(pr_table)

    sim = cirq.Simulator()
    sample_results = sim.sample(total_circuit, params=pr_table, repetitions=shots)

    # Results statistics
    sample_results = np.array(sample_results)
    # print(sample_results)

    results = dict()
    for sample in sample_results:
        sample_str = ""
        for q in sample[-n_qubits:]:
            sample_str += str(int(q))

        if sample_str not in results:
            results[sample_str] = 1
        else:
            results[sample_str] += 1

    print("Measurement result with Tensorflow Quantum:\n", results)


# Now run qaoa benchmark functions:

In [ ]:
import networkx as nx
import time
import matplotlib.pyplot as plt


def generate_graphs(n_nodes=4):
    """Generate a list containing random graphs generated by Networkx."""
    edges = []

    if n_nodes == 4:
        edges = [(0, 1), (0, 3), (1, 2), (2, 3)]
    elif n_nodes == 5:
        edges = [(0, 1), (1, 2), (2, 3), (3, 4), (0, 4), (0, 2)]
    elif n_nodes == 6:
        edges = [(0, 4), (0, 5), (1, 2), (1, 3), (1, 4), (1, 5), (2, 4), (3, 5)]
    elif n_nodes == 7:
        edges = [(0, 4), (0, 5), (0, 6), (1, 4), (1, 5), (2, 5), (2, 6), (3, 5), (3, 6)]
    elif n_nodes == 8:
        edges = [(1, 4), (1, 5), (1, 6), (2, 3), (2, 5), (2, 4), (3, 4), (3, 5), (3, 6), (3, 7), (4, 7),
                 (4, 5), (6, 2), (6, 7), (6, 5), (7, 0)]
    elif n_nodes == 10:
        edges = [(0, 4), (0, 2), (0, 9), (0, 8), (1, 6), (1, 5), (2, 5), (2, 6), (2, 4), (2, 8), (3, 4), (3, 8),
                 (3, 7), (3, 9), (6, 8), (7, 5), (7, 8), (7, 4), (7, 6), (8, 4), (9, 5), (9, 2)]
    elif n_nodes == 12:
        edges = [(0, 5), (0, 9), (0, 10), (0, 3), (1, 4), (2, 10), (2, 11), (2, 9), (2, 6), (2, 7), (3, 11), (3, 6),
                 (3, 7), (4, 7), (4, 5), (5, 7), (7, 8), (7, 9), (8, 10), (8, 9), (8, 11), (10, 11)]

    return nx.Graph(edges)


graph = generate_graphs(n_nodes=4)

nx.draw_networkx(graph)
plt.show()

hp = {'graph': graph,
      'n_layers': 6,
      'shots': 1000,
      'iter_num': 100}

mq_start = time.time()
benchmark_mq(hp)
mq_execution = time.time() - mq_start

# pd_start = time.time()
# benchmark_paddle(hp)
# pd_execution = time.time() - pd_start

qp_start = time.time()
benchmark_qpanda(hp)
qp_execution = time.time() - qp_start

pl_start = time.time()
benchmark_pennylane(hp)
pl_execution = time.time() - pl_start

qiskit_start = time.time()
benchmark_qiskit(hp)
qiskit_execution = time.time() - qiskit_start

tfq_start = time.time()
benchmark_tfq(hp)
tfq_execution = time.time() - tfq_start

print("Execution time with MindQuantum:", mq_execution)
# print("Execution time with PaddleQuantum:", pd_execution)
print("Execution time with QPanda:", qp_execution)
print("Execution time with PennyLane:", pl_execution)
print("Execution time with Qiskit:", qiskit_execution)
print("Execution time with Tensorflow Quantum:", tfq_execution)